In [1]:
import sys
import os
import env_var
import spark_session
import validate
import logging
import logging.config
import ingest
import data_prep

In [2]:
%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [3]:
open('application.log', 'w').close() ## To reset the application.log for every notebook run

In [4]:
logging.config.fileConfig('logging.config')
logger = logging.getLogger('root')
logger.setLevel(logging.DEBUG)

In [5]:
%time

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs


In [6]:
spark = spark_session.create_spark_session()

2025-02-26 19:30:40,674 - root - INFO - Creating Spark Session... create_spark_session Method started
2025-02-26 19:30:40,677 - root - INFO - Calling spark object...
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyteruser/.ivy2/cache
The jars for the packages stored in: /home/jupyteruser/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-969f9b32-649f-4c31-bbc5-2f5041a246a9;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 199ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   

2025-02-26 19:30:44,309 - root - INFO - Validating the Spark object


In [7]:
%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [8]:
file_format, header, schema, path_file = ingest.get_src_path(env_var.src_path,'csv')

In [9]:
%time

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.72 µs


In [10]:
main_df = ingest.read_data(in_spark=spark,in_file_path=path_file,in_file_format=file_format,in_header=header,in_schema=schema)

In [11]:
%time

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.58 µs


In [12]:
main_df.printSchema()

root
 |-- Open time: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Close time: timestamp (nullable = true)
 |-- Quote asset volume: double (nullable = true)
 |-- Number of trades: integer (nullable = true)
 |-- Taker buy base asset volume: double (nullable = true)
 |-- Taker buy quote asset volume: double (nullable = true)
 |-- Ignore: integer (nullable = true)



In [13]:
main_df = data_prep.renm_nd_chg_typ(main_df, env_var.bitcoin_cols)

In [14]:
%time

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


In [15]:
main_df.printSchema()

root
 |-- open_tm: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- vol: double (nullable = true)
 |-- close_tm: timestamp (nullable = true)
 |-- quote_asset_vol: float (nullable = true)
 |-- no_of_trades: integer (nullable = true)
 |-- taker_buy_base_asset_vol: float (nullable = true)
 |-- taker_buy_quote_asset_vol: float (nullable = true)
 |-- ignore: integer (nullable = true)



In [16]:
main_df =  data_prep.data_handling(main_df,date_column='open_tm', missing_strategy="drop",rolling_period=4,rolling_col='close')

In [17]:
main_df = data_prep.data_Partition(main_df, date_column='open_tm', partition_by='year')

In [18]:
main_df.printSchema()

root
 |-- open_tm: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- vol: double (nullable = true)
 |-- close_tm: timestamp (nullable = true)
 |-- quote_asset_vol: float (nullable = true)
 |-- no_of_trades: integer (nullable = true)
 |-- taker_buy_base_asset_vol: float (nullable = true)
 |-- taker_buy_quote_asset_vol: float (nullable = true)
 |-- ignore: integer (nullable = true)
 |-- rolling_avg: double (nullable = true)
 |-- year: integer (nullable = true)



In [19]:
main_df.rdd.getNumPartitions()

25/02/26 19:30:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
[Stage 11:

7

In [20]:
ingest.load_data(main_df, in_file_path=env_var.prep_path, partitioned_by='year', in_file_format='delta', in_mode='overwrite')

25/02/26 19:30:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 19:30:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/26 1

In [21]:
delta_df = ingest.create_delta_table(in_spark = spark, table_name = 'bitcoin', delta_table_path = env_var.prep_path)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|             open_tm|           timestamp|   NULL|
|                open|              double|   NULL|
|                high|              double|   NULL|
|                 low|              double|   NULL|
|               close|              double|   NULL|
|                 vol|              double|   NULL|
|            close_tm|           timestamp|   NULL|
|     quote_asset_vol|               float|   NULL|
|        no_of_trades|                 int|   NULL|
|taker_buy_base_as...|               float|   NULL|
|taker_buy_quote_a...|               float|   NULL|
|              ignore|                 int|   NULL|
|         rolling_avg|              double|   NULL|
|                year|                 int|   NULL|
|# Partition Infor...|                    |       |
|          # col_name|           data_type|comment|
|           

In [22]:
spark.version

'3.5.4'

In [23]:
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, env_var.prep_path):
    print(f"Delta table exists at {env_var.prep_path}")
else:
    print(f"No Delta table found at {env_var.prep_path}")

Delta table exists at /home/jupyteruser/work/data/prep_data/
